In [12]:
import warnings
warnings.filterwarnings('ignore')

import copy
import datetime
import glob
import h5py
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import sys

from cv2 import resize
from datetime import datetime
from gc import collect
from pandas.core.common import flatten
from os import cpu_count
from scipy.io import savemat, loadmat
from sklearn.model_selection import train_test_split
from time import sleep
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary
from tqdm import tqdm

sys.path.append(f"{os.getcwd()}/ViT architecture/working ViT")
sys.path.append(f"{os.getcwd()}/scripts/")
from VisionTransformer_working import VisionTransformer as Vit_old

sys.path.append(f"{os.getcwd()}/ViT architecture/Architecture tryouts/DPT/")
from VisionTransformer_working_for_DPT import VisionTransformer2 as Vit_2

In [2]:
random_seed = 1
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
cudnn.benchmark = True

In [3]:
collect()
torch.cuda.empty_cache()
device = torch.device('cpu')
print(f"Running on device: {device}")

Running on device: cpu


In [72]:
train_data_path = '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Training data/resolution_104/3750'
val_data_path = '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Validation data/resolution_104'
test_data_path = '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Test data/resolution_104'

In [75]:
train_dspl_paths = [] #to store image paths in list
train_trac_paths = [] #to store image paths in list

for file in os.listdir(train_data_path):
    if file.startswith('dspl'):
        train_dspl_paths.append(train_data_path + '/' + file)
    elif file.startswith('trac'):
        train_trac_paths.append(train_data_path + '/' + file)
    
train_dspl_paths = list(train_dspl_paths)
train_trac_paths = list(train_trac_paths)
train_dspl_paths.sort()
train_trac_paths.sort()

In [80]:
train_dspl_paths

['/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Training data/resolution_104/3750/dspl_21.h5',
 '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Training data/resolution_104/3750/dspl_22.h5',
 '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Training data/resolution_104/3750/dspl_23.h5',
 '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Training data/resolution_104/3750/dspl_24.h5',
 '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Training data/resolution_104/3750/dspl_25.h5',
 '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Training data/resolution_104/3750/dspl_26.h5',
 '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Training data/resolution_104/3750/dspl_27.h5',
 '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Training data/resolution_104/3750/dspl_28.h5',
 '/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Training data/resolution_104/3750/dspl_29.h5',
 '/home/alexrichard/PycharmProjects/UQ_DL-TFM/

In [12]:
from tables import *
h5file = h5py.File("/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM/data/Training data/resolution_104/3750/dspl_21.h5", mode="r")


In [14]:
h5file['data'].shape[0]

3750

In [16]:
class TFMDataset(Dataset):
    def __init__(self, dspl_path, trac_path):
        self.dspl_path = dspl_path
        self.trac_path = trac_path
        self.dataset = None
        with h5py.File(self.dspl_path, 'r') as file:
            self.dataset_len = file['data'].shape[0]
        
    def __len__(self):
        return self.dataset_len
    
    def __getitem__(self, index):
        if self.dataset is None:
            self.dspl_dataset = torch.moveaxis(h5py.File(self.dspl_path, 'r')["data"], 3, 1).float()
            self.trac_dataset = torch.moveaxis(h5py.File(self.trac_path, 'r')["data"], 3, 1).float()
            
        return self.dspl_dataset[index], self.trac_dataset[index]